In [27]:
#Import dependencies
import pandas as pd
import numpy as np
import requests
import time
import csv
import json
from pprint import pprint
from splinter import Browser 
from bs4 import BeautifulSoup as bs
import collections
import pymongo
from IPython.display import display_html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys



# Import API key
from api_keys import api_key


# Import the dataframe of all 2015 zipcodes and their respective precincts and sectors

In [28]:
df_zipPrct = pd.read_csv("Zipcodes and Precincts.csv")
df_zipPrct

,Zipcode,Precinct,Sector
0,10279,1,Sector C
1,10280,1,Sector B
2,10281,1,Sector C
3,10282,1,Sector C
4,10285,1,Sector C
5,10286,1,Sector A
6,10301,120,Sector B
7,10302,121,Sector B
8,10303,121,Sector A
9,10304,120,Sector D


In [29]:
#get a distinct list of precincts
all_precincts= list(set(list(df_zipPrct.Precinct)))
print(f"There are a total of {len(all_precincts)} precincts.")

There are a total of 73 precincts.


# TESTING calling json data and importing into dataframe

In [4]:
#TEST: query json nypd for precinct information, crime information, etc
"""
url = "https://data.cityofnewyork.us/resource/9s4h-37hy.json?"
#precinct query 
precinct = "addr_pct_cd="
#report date query
report_date = "rpt_dt="
year = 2015
month = 1
day = 17
x = 63
query_url = (f"{url}$$app_token={api_key}&{precinct}{x}&{report_date}{year}-{month}-{day}")
response = requests.get(query_url)
json_file = requests.get(query_url).json()
pprint(json_file)
"""

'\nurl = "https://data.cityofnewyork.us/resource/9s4h-37hy.json?"\n#precinct query \nprecinct = "addr_pct_cd="\n#report date query\nreport_date = "rpt_dt="\nyear = 2015\nmonth = 1\nday = 17\nx = 63\nquery_url = (f"{url}$$app_token={api_key}&{precinct}{x}&{report_date}{year}-{month}-{day}")\nresponse = requests.get(query_url)\njson_file = requests.get(query_url).json()\npprint(json_file)\n'

In [5]:
#TEST: create json file from data extracted from NYPD data json with api 
"""
results = []
for item in json_file:
    my_dict = {}
    my_dict['PRECINCT'] = item.get('addr_pct_cd')
    my_dict['TYPE'] = item.get('law_cat_cd')
    my_dict['DESCRIPTION'] = item.get('ofns_desc')
    latitude = item.get('lat_lon').get('coordinates')[1]
    longitude = item.get('lat_lon').get('coordinates')[0]
    my_dict['LAT_LONG'] = (float(latitude),float(longitude))
    pprint(my_dict)
    results.append(my_dict)

#back_json = json.dumps(results)
with open('test.json', 'w') as outfile:  
    json.dump(results, outfile)
"""

"\nresults = []\nfor item in json_file:\n    my_dict = {}\n    my_dict['PRECINCT'] = item.get('addr_pct_cd')\n    my_dict['TYPE'] = item.get('law_cat_cd')\n    my_dict['DESCRIPTION'] = item.get('ofns_desc')\n    latitude = item.get('lat_lon').get('coordinates')[1]\n    longitude = item.get('lat_lon').get('coordinates')[0]\n    my_dict['LAT_LONG'] = (float(latitude),float(longitude))\n    pprint(my_dict)\n    results.append(my_dict)\n\n#back_json = json.dumps(results)\nwith open('test.json', 'w') as outfile:  \n    json.dump(results, outfile)\n"

In [6]:
#test json file into dataframe
"""
df_json = pd.read_json("test.json")
df_json
"""

'\ndf_json = pd.read_json("test.json")\ndf_json\n'

# DEPLOY ABOVE FOR ALL 73 PRECINCTS

In [45]:
url = "https://data.cityofnewyork.us/resource/9s4h-37hy.json?"
#precinct query 
precinct = "addr_pct_cd="
#report date query
report_date = "rpt_dt="

def crime_search(year,x): 
    counter = 0
    totalPrecincts= len(all_precincts)
    for month in range(1,13):
        for day in range(1,32):
            query_url = (f"{url}$$app_token={api_key}&{precinct}{x}&{report_date}{year}-{month}-{day}")
            response = requests.get(query_url)
            text = str(response)
            json_file = requests.get(query_url).json()
            if text == "<Response [200]>":
                for item in json_file:
                    try:
                        my_dict = {}
                        my_dict['PRECINCT'] = item.get('addr_pct_cd')
                        my_dict['TYPE'] = item.get('law_cat_cd')
                        my_dict['DESCRIPTION'] = item.get('ofns_desc')
                        reportedDate = item.get('rpt_dt')
                        my_dict['REPORTED_DATE'] = reportedDate[0:10]
                        latitude = item.get('lat_lon').get('coordinates')[1]
                        longitude = item.get('lat_lon').get('coordinates')[0]
                        my_dict['LAT_LONG'] = (float(latitude),float(longitude))
                        results.append(my_dict)
                        counter += 1
                        print(f"Report {counter} from Precinct {x} for {month}/{day}/{year}. {totalPrecincts} precincts left...", end="\r")
                    except:pass
            elif text == "<Response [400]>": 
                counter += 0
    totalPrecincts-=1
    return print("Finished.")
    

In [46]:
results = []
for prc in all_precincts:
    crime_search(2015,prc)

with open('2015_CrimeData.json', 'w') as outfile:
    json.dump(results, outfile)

Finished.55 from Precinct 1 for 12/31/2015. 73 precincts left...
Finished.93 from Precinct 6 for 12/31/2015. 73 precincts left...
Finished.50 from Precinct 7 for 12/31/2015. 73 precincts left...
Finished.21 from Precinct 9 for 12/31/2015. 73 precincts left...
Finished.86 from Precinct 10 for 12/31/2015. 73 precincts left...
Finished.14 from Precinct 13 for 12/31/2015. 73 precincts left...
Finished.536 from Precinct 14 for 12/31/2015. 73 precincts left...
Finished.61 from Precinct 17 for 12/31/2015. 73 precincts left...
Finished.53 from Precinct 18 for 12/31/2015. 73 precincts left...
Finished.56 from Precinct 19 for 12/31/2015. 73 precincts left...
Finished.32 from Precinct 20 for 12/31/2015. 73 precincts left...
Finished.03 from Precinct 23 for 12/31/2015. 73 precincts left...
Finished.59 from Precinct 24 for 12/31/2015. 73 precincts left...
Finished.36 from Precinct 25 for 12/31/2015. 73 precincts left...
Finished.56 from Precinct 28 for 12/31/2015. 73 precincts left...
Finished.04 f

In [47]:
json_df = pd.read_json('2015_CrimeData.json')
json_df
#json_df = pd.read_json('test_output.json')
#json_df
#json_df.to_csv("testagain.csv", sep=',')

,DESCRIPTION,LAT_LONG,PRECINCT,REPORTED_DATE,TYPE
0,BURGLARY,"[40.725585762, -74.000499329]",1,2015-01-01,FELONY
1,GRAND LARCENY,"[40.711699719, -74.008348243]",1,2015-01-01,FELONY
2,PETIT LARCENY,"[40.710093847, -74.01060963]",1,2015-01-01,MISDEMEANOR
3,PETIT LARCENY,"[40.720999167, -74.004922198]",1,2015-01-01,MISDEMEANOR
4,GRAND LARCENY,"[40.705639013, -74.011452902]",1,2015-01-01,FELONY
5,CRIMINAL MISCHIEF & RELATED OF,"[40.72046384, -74.006852204]",1,2015-01-01,MISDEMEANOR
6,ASSAULT 3 & RELATED OFFENSES,"[40.70780671, -74.016874407]",1,2015-01-02,MISDEMEANOR
7,BURGLARY,"[40.727056937, -74.001895604]",1,2015-01-02,FELONY
8,ASSAULT 3 & RELATED OFFENSES,"[40.720255223, -74.00709028]",1,2015-01-02,MISDEMEANOR
9,GRAND LARCENY,"[40.724441186, -73.998186736]",1,2015-01-02,FELONY


In [ ]:
print(crime_search())

In [ ]:
#counts the number of police reports by year and precinct
def crime_search(year,x):
    #makes 'counter' into a global variable that can be used outside of the function
    global counter
    counter = 0 
    #loops through every month of the year
    for month in range(1,13):
        for day in range(1,32):
            query_url = (f"{url}$$app_token={api_key}&{precinct}{x}&{report_date}{year}-{month}-{day}")
            response = requests.get(query_url)
            text = str(response)
            json = requests.get(query_url).json()
            if text == "<Response [200]>":
                for entry in json:
                    counter += 1
                    # end ="\r" makes it so that below processing log prints over the same line
                    print(f"Processing Report {counter}", end="\r"),
            elif text == "<Response [400]>": 
                counter += 0
    return counter